In [2]:
import requests
import pandas as pd
import numpy as np

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

import folium

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [3]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

url_sydney = "https://en.wikipedia.org/wiki/City_of_Sydney"
soup_sydney = BeautifulSoup(requests.get(url_sydney).text, 'html.parser')

results = soup_sydney.find_all('div', attrs={'class':'div-col columns column-width'})[0].find_all('a', href=True)
# results

In [4]:
suburbs_sydney = []
for result in results:
    suburbs_sydney.append(result.get('title'))
# suburbs_sydney
df_suburbs_sydney=pd.DataFrame(suburbs_sydney, columns=['address'])
# df_suburbs_sydney.loc[(~df_suburbs_sydney['address'].str.contains('New South Wales')),'address'] +=', New South Wales'
df_suburbs_sydney['address'] = df_suburbs_sydney['address'].str.replace(', New South Wales','')
df_suburbs_sydney=df_suburbs_sydney.replace(regex=r'central business district', value='CBD')
df_suburbs_sydney['address'] += ', Sydney'
df_suburbs_sydney[['Suburb','Region']] = df_suburbs_sydney.address.str.split(", ", expand=True,)
df_suburbs_sydney.drop(df_suburbs_sydney[df_suburbs_sydney['Suburb'].str.contains('Council|Randwick|Woollahra')].index,inplace=True)

df_suburbs_sydney.head()


,address,Suburb,Region
0,"Alexandria, Sydney",Alexandria,Sydney
1,"Annandale, Sydney",Annandale,Sydney
3,"Barangaroo, Sydney",Barangaroo,Sydney
4,"Beaconsfield, Sydney",Beaconsfield,Sydney
5,"Camperdown, Sydney",Camperdown,Sydney


In [5]:
# get geolocations of suburbs

# latitudes =[]
# longitudes =[]

# for address in df_suburbs_sydney['address']:
#     geolocator = Nominatim(user_agent="sydney_explorer")
#     geolocator=Nominatim(timeout=10)
#     location = geolocator.geocode(address)
#     # print(location)
#     latitude = location.latitude
#     longitude = location.longitude
#     # print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
#     latitudes.append(latitude)
#     longitudes.append(longitude)

# print(latitudes)
# print(longitudes)

latitudes = [-33.9091568, -33.881224, -33.8614084, -33.9114691, -33.8896116, -33.8977778, -33.8863291, -33.8783381, -33.890862, -33.8572223, -33.8716908, -33.9021718, -33.895, -33.8805556, -33.8777782, -33.8814408, -33.8599129, -33.8958333, -33.8961132, -33.8841572, -33.8670797, -33.8692145, -33.8931044, -33.9185861, -33.8750365, -33.9159472, -33.8845123, -33.8679574, -33.859992, -33.8794728, -33.9002762, -33.8718758, -33.9076623]
longitudes = [151.1921281, 151.1709976, 151.2016877, 151.200315, 151.1800986, 151.2338889, 151.1998211, 151.219225, 151.1932156, 151.2067756, 151.2282875, 151.1859881, 151.1913889, 151.1783333, 151.1862377, 151.2044524, 151.2041274, 151.2219444, 151.1801893, 151.2272766, 151.2259967, 151.1923633, 151.2040292, 151.204692, 151.2276557, 151.176332, 151.2100301, 151.210047, 151.2083216, 151.1984346, 151.2073137, 151.2192218, 151.2082183]


In [6]:
df_suburbs_sydney=df_suburbs_sydney.drop(columns=['address'])

In [7]:
df_suburbs_sydney['Latitude'] = latitudes
df_suburbs_sydney['Longitude'] = longitudes
df_suburbs_sydney['Region'] = 'City of Sydney'
df_suburbs_sydney=df_suburbs_sydney.reset_index(drop=True)
df_suburbs_sydney.to_csv('suburbs_sydney.csv')
df_suburbs_sydney.head()

,Suburb,Region,Latitude,Longitude
0,Alexandria,City of Sydney,-33.909157,151.192128
1,Annandale,City of Sydney,-33.881224,151.170998
2,Barangaroo,City of Sydney,-33.861408,151.201688
3,Beaconsfield,City of Sydney,-33.911469,151.200315
4,Camperdown,City of Sydney,-33.889612,151.180099


In [8]:
address = 'city of Sydney, Australia'

geolocator = Nominatim(user_agent="sydney_explorer")
location = geolocator.geocode(address)
sydney_latitude = location.latitude
sydney_longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(sydney_latitude, sydney_longitude))

# create map of sydney using latitude and longitude values
map_sydney = folium.Map(location=[sydney_latitude, sydney_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_suburbs_sydney['Latitude'], df_suburbs_sydney['Longitude'], df_suburbs_sydney['Region'], df_suburbs_sydney['Suburb']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)

map_sydney

The geograpical coordinate of Sydney are -33.8853222, 151.2065221.
